## Task 1 - Data Loading and Preprocessing

In [56]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np
import pandas as pd
import torch
df=pd.read_csv('Iris.csv')
df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


In [57]:
X = df.drop('Id', axis=1).iloc[:, :-1].values # features
y = df.iloc[:, -1].values # target
le = LabelEncoder()
y = le.fit_transform(y)

In [58]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)
scaler= StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [59]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
print(y_train.shape)
print(y_train[:5])

(120, 3)
[[1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


## Task 2 - Neural Network Construction

In [60]:
import torch.nn as nn
import torch.nn.functional as F

class IrisNet(nn.Module):
    def __init__(self):
        super(IrisNet, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 8) # Input layer to hidden layer
        self.fc2 = nn.Linear(8, 3) # Hidden layer to output layer

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)# softmax function applied to the output layer

model = IrisNet()
print(model)

IrisNet(
  (fc1): Linear(in_features=4, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=3, bias=True)
)


## Task 3 - Model Compilation and Training

In [61]:
import torch.optim as optim
import torch.nn as nn
import torch

# the Loss function and optimizer
criterion = nn.CrossEntropyLoss() # For multi-class classification with one-hot encoded labels
optimizer = optim.Adam(model.parameters(), lr=0.001)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


epochs = 100
batch_size = 5
train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

print("Finished Training")

Epoch 1/100, Loss: 1.1370
Epoch 2/100, Loss: 1.1171
Epoch 3/100, Loss: 1.0992
Epoch 4/100, Loss: 1.0823
Epoch 5/100, Loss: 1.0636
Epoch 6/100, Loss: 1.0428
Epoch 7/100, Loss: 1.0186
Epoch 8/100, Loss: 0.9921
Epoch 9/100, Loss: 0.9638
Epoch 10/100, Loss: 0.9351
Epoch 11/100, Loss: 0.9082
Epoch 12/100, Loss: 0.8861
Epoch 13/100, Loss: 0.8675
Epoch 14/100, Loss: 0.8522
Epoch 15/100, Loss: 0.8405
Epoch 16/100, Loss: 0.8307
Epoch 17/100, Loss: 0.8223
Epoch 18/100, Loss: 0.8148
Epoch 19/100, Loss: 0.8084
Epoch 20/100, Loss: 0.8017
Epoch 21/100, Loss: 0.7956
Epoch 22/100, Loss: 0.7895
Epoch 23/100, Loss: 0.7840
Epoch 24/100, Loss: 0.7780
Epoch 25/100, Loss: 0.7724
Epoch 26/100, Loss: 0.7668
Epoch 27/100, Loss: 0.7616
Epoch 28/100, Loss: 0.7556
Epoch 29/100, Loss: 0.7507
Epoch 30/100, Loss: 0.7453
Epoch 31/100, Loss: 0.7403
Epoch 32/100, Loss: 0.7352
Epoch 33/100, Loss: 0.7303
Epoch 34/100, Loss: 0.7257
Epoch 35/100, Loss: 0.7211
Epoch 36/100, Loss: 0.7169
Epoch 37/100, Loss: 0.7125
Epoch 38/1

## Task 4 - Model Evaluation

In [62]:
model.eval() # Set the model to evaluation mode
with torch.no_grad(): # Disable gradient calculation
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs.data, 1)
    _, true_labels = torch.max(y_test_tensor.data, 1)
    correct = (predicted == true_labels).sum().item()
    total = true_labels.size(0)
    accuracy = 100 * correct / total

print(f'Accuracy of the network on the {total} test samples: {accuracy:.2f} %')

Accuracy of the network on the 30 test samples: 96.67 %
